In [116]:
#start rafiki
!bash ./scripts/start.sh

---------------------------------------
| Creating Docker swarm for Rafiki... |
---------------------------------------
Error response from daemon: This node is already part of a swarm. Use "docker swarm leave" to leave this swarm and join another one.
Failed to init Docker swarm - continuing...
Error response from daemon: network with name rafiki already exists
Failed to create Docker network for swarm - continuing...
------------------------------------
| Creating Rafiki's logs folder... |
------------------------------------
Created Rafiki's logs directory!
------------------------------------------------
| Pulling images for Rafiki from Docker Hub... |
------------------------------------------------
Pulling images required by Rafiki from Docker Hub...
postgres:10.5 already exists locally
redis:5.0-rc already exists locally
rafikiai/rafiki_admin:0.0.4 already exists locally
rafikiai/rafiki_advisor:0.0.4 already exists locally
rafikiai/rafiki_worker:0.0.4 already exists locally
rafi

In [115]:
#stop rafiki
!bash ./scripts/stop.sh

---------------------------
| Stopping Rafiki's DB... |
---------------------------
rafiki_db
------------------------------
| Stopping Rafiki's Cache... |
------------------------------
rafiki_cache
------------------------------
| Stopping Rafiki's Admin... |
------------------------------
rafiki_admin
--------------------------------
| Stopping Rafiki's Advisor... |
--------------------------------
rafiki_advisor
You'll need to destroy your machine's Docker swarm & Rafiki's logs folder at /home/wubiao/log/rafiki manually


In [70]:
!cat .env.sh

export POSTGRES_HOST=rafiki_db
export POSTGRES_PORT=5432
export POSTGRES_USER=rafiki
export POSTGRES_DB=rafiki
export POSTGRES_PASSWORD=rafiki
export DOCKER_NETWORK=rafiki
#export LOGS_FOLDER_PATH=/var/log/rafiki
export LOGS_FOLDER_PATH=/home/wubiao/log/rafiki
export ADMIN_HOST=rafiki_admin
export ADMIN_PORT=8000
export ADVISOR_HOST=rafiki_advisor
export ADVISOR_PORT=8001
export REDIS_HOST=rafiki_cache
export REDIS_PORT=6379
export PREDICTOR_PORT=8002
export RAFIKI_IP_ADDRESS=127.0.0.1
export RAFIKI_IMAGE_ADMIN=rafikiai/rafiki_admin
export RAFIKI_IMAGE_ADVISOR=rafikiai/rafiki_advisor
export RAFIKI_IMAGE_WORKER=rafikiai/rafiki_worker
export RAFIKI_IMAGE_PREDICTOR=rafikiai/rafiki_predictor
export IMAGE_POSTGRES=postgres:10.5
export IMAGE_REDIS=redis:5.0-rc
export RAFIKI_VERSION=0.0.4
export PYTHONPATH=$PWD


In [117]:
#start client
from rafiki.client import Client
client = Client(admin_host='localhost', admin_port=8000)
client.login(email='superadmin@rafiki', password='rafiki')

{'user_id': 'f86b4b2c-891a-4f9f-a221-a761c237f5eb', 'user_type': 'SUPERADMIN'}

In [ ]:
#add models for default IMAGE_CLASSIFICATION
client.create_model(
    name='TfSingleHiddenLayer',
    task='IMAGE_CLASSIFICATION',
    model_file_path='examples/models/image_classification/TfSingleHiddenLayer.py',
    model_class='TfSingleHiddenLayer'
)

client.create_model(
    name='SkDt',
    task='IMAGE_CLASSIFICATION',
    model_file_path='examples/models/image_classification/SkDt.py',
    model_class='SkDt'
)

#check loaded models by task
client.get_models_of_task(task='IMAGE_CLASSIFICATION')

In [118]:
#add model for FEATURE_VECTOR_CLASSIFICATION
client.create_model(
    name='SkDtVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkDt.py',
    model_class='SkDtVector'
)

#check loaded models by task
client.get_models_of_task(task='FEATURE_VECTOR_CLASSIFICATION')

[{'datetime_created': 'Sun, 28 Oct 2018 10:57:55 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.4',
  'model_class': 'SkDtVector',
  'name': 'SkDtVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': 'f86b4b2c-891a-4f9f-a221-a761c237f5eb'}]

uri = "https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true"

In [120]:
#create a training job
client.create_train_job(
    app='concept_drift',
    task='FEATURE_VECTOR_CLASSIFICATION',
    train_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true',
    test_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_test.zip?raw=true',
    budget_type='MODEL_TRIAL_COUNT',
    budget_amount=2
)

{'app': 'concept_drift',
 'app_version': 1,
 'id': '9fb15870-27bd-46ba-bd66-6b84d2ae1276'}

In [129]:
#get train job status
client.get_train_job(app='concept_drift')

Exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/root/rafiki/rafiki/utils/auth.py", line 38, in wrapped
    return f(auth, *args, **kwargs)
  File "/root/rafiki/rafiki/admin/app.py", line 73, in get_train_job
    return jsonify(admin.get_train_job(app, app_version=int(app_version), **params))
  File "/root/rafiki/rafiki/admin/admin.py", line 116, in get_train_job
    train_job = self._db.get_train_job_by_app_version(app, app_version=app_version)
  File "/root/rafiki/rafiki/db/database.py", line 97, in get_train_job_by_app_version
    return query.first()
  File "/usr/local/lib/python3.6/site-packages/sqlalchemy/orm/query.py", line 2888, in first
    ret = list(self[0:1])
  File "/usr/local/lib/pyth

In [ ]:
#get job metrics
client.get_best_trials_of_train_job(app='concept_drift')

In [ ]:
#startinference job
client.create_inference_job(app='concept_drift')

In [ ]:
#stop inference job
client.stop_inference_job(app='concept_drift')

In [126]:
from rafiki.model import load_dataset
from rafiki.constants import TaskType
train_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true'
train, test = load_dataset(train_dataset_uri, TaskType.FEATURE_VECTOR_CLASSIFICATION)

AttributeError: type object 'TaskType' has no attribute 'FEATURE_VECTOR_CLASSIFICATION'

In [128]:
TaskType.IMAGE_CLASSIFICATION

'IMAGE_CLASSIFICATION'